# AI Meeting Recorder Agent Setup

This notebook demonstrates how to register an AI Meeting Recorder Agent with Xpander Cloud. The agent is designed to record meetings, check recording status, send email notifications, and create calendar events.

## Prerequisites

Before running this notebook, ensure you have:
1. An Xpander Cloud account with API access
2. A `.env` file containing your `XPANDER_API_KEY`
3. Required Python packages installed

## 1. Setup and Environment Configuration

First, let's import the required libraries and load our environment variables.

In [2]:
"""AI Meeting Recorder Agent registration script for Xpander Cloud."""
import os
import json
from xpander_sdk import XpanderClient, GraphItem, AgentGraphItemType
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API key is available
xpander_api_key = os.environ.get("XPANDER_API_KEY")
if not xpander_api_key:
    print("❗️ Missing XPANDER_API_KEY in .env file")
else:
    print("✅ XPANDER_API_KEY loaded successfully")

# Initialize Xpander client
xpander_client = XpanderClient(api_key=xpander_api_key)

✅ XPANDER_API_KEY loaded successfully


## 2. Load Agent Configuration

The agent configuration is stored in a JSON file that contains the agent name and the tools it needs to function.

In [3]:
# Load configuration from file
config_path = "agent_config.json"  # You can change this to a custom path

try:
    with open(config_path, "r") as f:
        config = json.load(f)
    
    agent_name = config.get("agent_name")
    tools = config.get("tools", [])
    print(f"Using config file: {config_path}")
    print(f"Agent name: {agent_name}")
    print(f"Number of tools: {len(tools)}")
    
    # Display the tools configuration
    for i, tool in enumerate(tools, 1):
        print(f"\nTool {i}:")
        print(f"  Interface: {tool.get('interface')}")
        print(f"  Internal Name: {tool.get('internal_name')}")
        print(f"  Path: {tool.get('path')}")
except (FileNotFoundError, json.JSONDecodeError) as e:
    print(f"❌ Error loading config: {e}")

Using config file: agent_config.json
Agent name: AI Meeting Recorder Agent
Number of tools: 4

Tool 1:
  Interface: xpanderai tools
  Internal Name: XpanderMeetingManagementCreateRecordingBot
  Path: /business/meeting/create_recorder

Tool 2:
  Interface: xpanderai tools
  Internal Name: XpanderMeetingManagementCheckRecorderStatus
  Path: /business/meeting/status/{recorder_id}

Tool 3:
  Interface: xpanderai tools
  Internal Name: XpanderMessagingServiceSendEmailWithContent
  Path: /messaging/send_email

Tool 4:
  Interface: google-calendar
  Internal Name: CalendarEventManagementCreateEventInCalendarById
  Path: /calendars/{calendarId}/events


## 3. Get or Create Agent

This section checks if an agent ID exists in the environment variables. If it does, we use the existing agent. If not, we create a new one and update the `.env` file with the new agent ID.

In [ ]:
# Get existing agent ID from environment or create a new agent
agent_id = os.environ.get("XPANDER_AGENT_ID")

try:
    if agent_id:
        agent = xpander_client.agents.get(agent_id=agent_id)
        print(f"Using existing agent: {agent.name} (ID: {agent.id})")
    else:
        agent = xpander_client.agents.create(name=agent_name)
        agent_id = agent.id
        
        # Update .env with new agent ID ## Optional
        with open(".env", "r") as f:
            env_content = f.read()
        with open(".env", "w") as f:
            if "XPANDER_AGENT_ID" in env_content:
                f.write("\n".join([line if not line.startswith("XPANDER_AGENT_ID=") 
                                 else f"XPANDER_AGENT_ID={agent.id}" for line in env_content.splitlines()]))
            else:
                f.write(f"{env_content}\nXPANDER_AGENT_ID={agent.id}")
        print(f"Created new agent: {agent.name} (ID: {agent.id})")
except Exception as e:
    print(f"❌ Error with agent: {e}")

Created new agent: AI Meeting Recorder Agent (ID: 6bb96105-3675-4d79-9dd3-8ae146cff007)


## 4. Retrieve Available Interfaces

Here we retrieve all the agentic interfaces available in the Xpander tenant. These interfaces contain the operations we can add as tools to our agent.

In [18]:
agent = xpander_client.agents.get(agent_id=agent_id)

# Retrieve available interfaces
try:
    interfaces = {}
    for interface in agent.retrieve_agentic_interfaces():
        name = getattr(interface, 'name', '').lower()
        interfaces[name] = interface
    
    print(f"Found {len(interfaces)} available interfaces:")
    for name in interfaces.keys():
        print(f"  - {name}")
except Exception as e:
    print(f"Error retrieving interfaces: {e}")
    # For demonstration, creating some dummy interfaces
    print("\nFor demonstration, using example interfaces:")
    print("  - xpanderai tools")
    print("  - google-calendar")

Found 24 available interfaces:
  - company monitor
  - social media
  - movie database
  - arxiv
  - zillow
  - weather
  - firecrawl
  - hackernews
  - similarweb
  - live sports data
  - midjourney
  - market insights
  - reddit
  - amazon retail data
  - xpanderai tools
  - fibery-demo
  - google-drive-sandbox
  - google-sheets-sandbox
  - google-calendar-sandbox
  - gmail-sandbox
  - notion-sandbox
  - xpander-datadog
  - xpander-github-production
  - xpander sandbox's personal slack


## 5. Get Existing Tools

Check which tools are already attached to the agent to avoid duplicates.

In [19]:
# Get existing tools
existing_tools = {}
try:
    if 'agent' in locals():
        tools_list = agent.get_tools() or []
        for tool in tools_list:
            if isinstance(tool, dict) and 'function' in tool and isinstance(tool['function'], dict):
                fn = tool['function']
                name = fn.get('name')
                if name:
                    existing_tools[name] = True
        print(f"Found {len(existing_tools)} existing tools:")
        for name in existing_tools.keys():
            print(f"  - {name}")
except Exception as e:
    print(f"Error getting existing tools: {e}")

Found 1 existing tools:
  - xpfinish-agent-execution-finished


## 6. Process Tools from Configuration

This section identifies which tools from our configuration need to be added to the agent.

In [20]:
# Process tools from config
to_add = []
missing = []

if 'tools' in locals() and 'interfaces' in locals() and 'agent' in locals():
    for tool_config in tools:
        interface_name = tool_config.get("interface", "").lower()
        internal_name = tool_config.get("internal_name", "")
        path = tool_config.get("path", "")
        
        if not internal_name:
            print(f"⚠️ Missing internal_name in tool config")
            continue
            
        if internal_name in existing_tools:
            print(f"✓ Tool already exists: {internal_name}")
            continue
        
        # Find interface (case insensitive partial match)
        interface = None
        for name, intf in interfaces.items():
            if interface_name in name:
                interface = intf
                print(f"Found interface: {name}")
                break
                
        if not interface:
            print(f"⚠️ Interface not found: {interface_name}")
            missing.append(internal_name)
            continue
            
        try:
            operations = agent.retrieve_agentic_operations(agentic_interface=interface)
            
            # Try to find by path if available in config
            operation = None
            if path:
                operation = next((op for op in operations if hasattr(op, 'path') and op.path == path), None)
                if operation:
                    print(f"+ Found tool by path: {path}")
                    print(f"+ Will add tool: {operation.name}")
                    to_add.append(operation)
                else:
                    print(f"❌ Operation not found with path: {path}")
                    missing.append(internal_name)
            else:
                print(f"❌ No path specified for: {internal_name}")
                missing.append(internal_name)
        except Exception as e:
            print(f"Error retrieving operations: {e}")
            missing.append(internal_name)
else:
    print("Skipping tool processing for demonstration purposes")
    print("In a real run, this would identify tools to add and missing tools")

Found interface: xpanderai tools
+ Found tool by path: /business/meeting/create_recorder
+ Will add tool: Create Meeting Recording Bot
Found interface: xpanderai tools
+ Found tool by path: /business/meeting/status/{recorder_id}
+ Will add tool: Check Recorder Status
Found interface: xpanderai tools
+ Found tool by path: /messaging/send_email
+ Will add tool: Send Email with Content
Found interface: google-calendar-sandbox
+ Found tool by path: /calendars/{calendarId}/events
+ Will add tool: Create Event in Calendar by ID


## 7. Add Tools to Agent

Finally, we add the identified tools to the agent and update its graph representation.

In [22]:
# Add tools if needed
if to_add:
    try:
        print(f"adding {len(to_add)} tools to the agent")
        agent.attach_operations(operations=to_add)
        for op in to_add:
            agent.graph.add_node(GraphItem(
                agent=agent, item_id=op.id_to_use_on_graph, 
                name=op.name, type=AgentGraphItemType.TOOL, is_local_tool=False
            ))
        agent.sync()
        print(f"✅ Successfully added {len(to_add)} tools")
    except Exception as e:
        print(f"❌ Error adding tools: {e}")
else:
    print("No tools to add or skipping for demonstration")

adding 4 tools to the agent
✅ Successfully added 4 tools


## 8. Summary

Let's print a summary of what we've done and provide links to the agent in the Xpander dashboard.

In [25]:
# Print summary
print(f"\nAgent setup complete!")
print(f"Go to: https://app.xpander.ai/agents/{agent.id}")

if missing:
    print("\nSome tools were not found. To find the exact internal names and paths:")
    print("python find_operations.py <search_term> --format config --output new_config.json")


Agent setup complete!
Go to: https://app.xpander.ai/agents/6bb96105-3675-4d79-9dd3-8ae146cff007


## Next Steps

Once your agent is set up with the appropriate tools, you can:

1. Test the agent by creating a meeting recording
2. Monitor recording status
3. Set up email notifications for completed recordings
4. Schedule follow-up calendar events

Visit the [Xpander documentation](https://docs.xpander.ai) for more information on working with agents and tools.